In [3]:
#default_exp ghtop

# ghtop

> I cannot believe this library name is not already taken on pypi.

In [4]:
#export
import time, sys, signal, shutil, os, json, enlighten, emoji, blessed
from ghtop.dashing import *
from collections import defaultdict

from fastcore.utils import *
from ghapi.all import *

In [5]:
term = Terminal()
logfile = Path("log.txt")

In [6]:
def github_auth_device():
    auth = GhDeviceAuth()
    wb = input(f"""First copy your one-time code: {term.yellow}{auth.user_code}{term.normal}
    Then visit {auth.verification_uri} in your browser, and paste the code when prompted.
    Shall we try to open the link for you? [y/n] """)
    if wb[0].lower()=='y': auth.open_browser()

    print ("Waiting for authorization...", end='')
    token = auth.wait(lambda: print('.', end=''))
    print("Authenticated with GitHub!")
    return token

In [7]:
#export
def exit(msg):
    print(msg, file=sys.stderr)
    sys.exit()

In [8]:
#export
def get_token():
    path = Path.home()/".ghtop_token"
    if path.is_file():
        try: return path.read_text().strip()
        except: exit("Error reading token")

    token = github_auth_device()
    path.write_text(token)
    return token

In [9]:
token = get_token()

In [39]:
def limit_cb(rem,quota):
    if int(rem) >= 1000: return
    print("WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING ")
    print("Remaining calls: " + str(remaining_apis))
    print("WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING ")

In [40]:
api = GhApi(limit_cb=limit_cb)

In [41]:
#export
def fetch_events(): return api.activity.list_public_events(per_page=100)

In [42]:
#export
def read_json_log():
    try: return logfile.read_json()
    except FileNotFoundError: return []

In [85]:
# EVENT_EMOJI_MAPPING = defaultdict(str, dict(
#     CreateEvent='🆕',
#     DeleteEvent='❌',
#     DeploymentEvent='🚀',
#     ForkEvent='🍴',
#     GistEvent='📒',
#     IssueCommentEvent='💬',
#     LabelEvent='🏷',
#     PushEvent='🤜',
#     ReleaseEvent='🚀',
#     StarEvent='⭐',
#     WatchEvent='👀'
# ))

In [86]:
Events = dict(
    IssuesEvent_closed=('⭐', 'closed', noop, 'issue'),
    IssuesEvent_opened=('📫', 'opened', noop, 'issue'),
    IssueCommentEvent=('💬', 'commented on', term.white, 'issue'),
    PullRequestEvent_opened=('✨', 'opened a pull request', term.yellow, 'pull_request'),
    PullRequestEvent_closed=('✔', 'closed a pull request', term.green, 'pull_request'),
)

In [87]:
printed_event_ids = {}

In [88]:
def _title(d): return f'("{d.title[:50]}...")'
def _repo(r,i=20): return f'on repo {r[:i]}'
def _release(e, login, repo): return f'🚀 {login} released {e.payload.release.tag_name} of {repo}'

In [92]:
def to_log(e):
    login,repo,pay = e.actor.login,e.repo.name,e.payload
    typ = e.type + (f'_{pay.action}' if e.type in ('PullRequestEvent','IssuesEvent') else '')
    emoji,msg,color,attr = Events.get(typ, [0]*4)
    if emoji:
        xtra = '' if e.type == "PullRequestEvent" else f' issue # {pay.issue.number}'
        return color(f'{emoji} {login} {msg}{xtra} {_repo(repo)} {_title(pay[attr])}')
    elif e.type == "ReleaseEvent": return term.firebrick3(_release(e, login, repo))

In [93]:
#export
def print_event(e, commits_counter):
    if e.id in printed_event_ids: return
    printed_event_ids[e.id] = 1
    login = e.actor.login
    if "bot" in login or "b0t" in login: return  # Don't print bot activity (there is a lot!)

    res = to_log(e)
    if res: print(res)
    elif e.type == "PushEvent": [commits_counter.update() for c in e.payload.commits]
    elif e.type == "SecurityAdvisoryEvent": print(term.blink("SECURITY ADVISORY"))

In [94]:
ev = [o for o in fetch_events() if o.type in ('IssuesEvent','ReleaseEvent','PullRequestEvent')]
[(i,o.type) for i,o in enumerate(ev)]
printed_event_ids={}
for e in ev: print_event(e,None)

✔ lacomaco closed a pull request on repo boostcamp-2020/Proje ("PieGraph 페이지 생성...")
✨ ddacw opened a pull request on repo CS201-Platal-Game/Pl ("Clock implementation...")
✔ dangokanako closed a pull request on repo dangokanako/JieLink. ("获取最新代码？...")
🚀 tuuzed released 0.12.0.BETA of tuuzed/LightTunnel
✔ OrionEdwards-Gallagher closed a pull request on repo GallagherSecurity/mo ("Orion/sdk15...")
📫 HCookY95 opened issue # 6 on repo XiuweiHe/EmotionClas ("Unknow url type...")
✔ stephqq closed a pull request on repo stephqq/thinkMindful ("Bump lodash from 4.17.15 to 4.17.19...")
✨ rachellegelden opened a pull request on repo canvas-gamification/ ("cleanup exam workflow...")
📫 Yamboy1 opened issue # 8704 on repo denoland/deno ("Dead link on `Script Installer` manual page...")
✔ r2dev2bb8 closed a pull request on repo KentoNishi/LiveTL ("Safari Auto Build now pulls version number automat...")


In [72]:
#export    
def write_logs(events):
    Path("tmp.log").write_text(json.dumps(events, indent=2))
    shutil.move("tmp.log", logfile)

def tail_events():
    manager = enlighten.get_manager()
    commits = manager.counter(desc='Commits', unit='commits', color='green')
    while True:
        events = fetch_events()
        log = read_json_log()
        combined = sorted(log + events, key=lambda x: int(x["id"]))
        write_logs(combined)
        for x in combined: print_event(x, commits)
        time.sleep(0.2)

In [73]:
combined = (fetch_events()+read_json_log()).sorted(using_attr(int,"id"))

In [76]:
#export
def watch_users():
    users,users_events = defaultdict(int),defaultdict(lambda: defaultdict(int))
    while True:
        for x in fetch_events():
            login = e.actor.login
            users[login] += 1
            users_events[login][x.type] += 1

        print (term.clear())
        print ("User".ljust(30), "Events".ljust(6), "PRs".ljust(5), "Issues".ljust(6), "Pushes".ljust(7))
        sorted_users = sorted(users.items(), key = lambda kv: (kv[1], kv[0]), reverse=True)
        for i in range(20):
            u = sorted_users[i]
            ue = users_events[u[0]]
            print(u[0].ljust(30), str(u[1]).ljust(6), 
                  str(ue.get('PullRequestEvent', '')).ljust(5), 
                  str(ue.get('IssuesEvent', '')).ljust(6), 
                  str(ue.get('PushEvent', '')).ljust(7))
        time.sleep(1)

In [77]:
#export
def push_to_log(e):
    return f"{e.actor.login} pushed {len(e.payload.commits)} commits to repo {e.repo.name}"

In [81]:
#export
def str_clean(s): return s[:95]

In [ ]:
def signal_handler(sig, frame):
    if sig != signal.SIGINT: return
    print(term.exit_fullscreen())
    print(term.clear())
    print(term.normal)
    sys.exit(0)

In [ ]:
#export
def quad_logs():
    term.enter_fullscreen()
    ui = HSplit(
            VSplit(
                Log(title='Issues', border_color = 2, color=7),
                Log(title='Commits', border_color = 2, color=3)
            ),
            VSplit(
                Log(title='Pull Requests', border_color = 2, color=4),
                Log(title='Releases', border_color = 2, color=5)
            ),
        )

    issues,commits = ui.items[0].items
    prs,releases = ui.items[1].items
    for o in issues,commits,prs,releases: o.append(" ")

    while True:
        for x in fetch_events():
            t = x["type"]
            if t == 'PushEvent': commits.append(str_clean(push_to_log(x)))
            elif t == 'IssuesEvent' or t == 'IssueCommentEvent': issues.append(str_clean(to_log(x)))
            elif t == 'PullRequestEvent': prs.append(str_clean(to_log(x)))
            elif t == 'ReleaseEvent': releases.append(str_clean(to_log(x)))
        ui.display()
        time.sleep(0.1)

In [ ]:
#export
def simple():
    while True:
        for x in fetch_events(): print(f"{x.actor.login} {x.type} {x.repo.name}")

In [ ]:
if len(sys.argv) < 2: exit("Usage: ghtop <tail|quad|users|simple>")
signal.signal(signal.SIGINT, signal_handler)

if sys.argv[1] == 'tail': tail_events()
elif sys.argv[1] == 'quad': quad_logs()
elif sys.argv[1] == 'users': watch_users()
elif sys.argv[1] == 'simple': simple()